In [15]:
# see 'requirements.txt' for required modules

import requests
from pprint import pprint
import os

from azure.ai.textanalytics import TextAnalyticsClient, TextAnalyticsApiKeyCredential

In [16]:
key = "f4a3533e6eb8431d9dca52beb055c8f5"
endpoint = "https://sotu.cognitiveservices.azure.com/"

def authenticate_client(k_tx, e_tx):
    ta_credential = TextAnalyticsApiKeyCredential(k_tx)
    text_analytics_client = TextAnalyticsClient(
            endpoint=e_tx, credential=ta_credential)
    return text_analytics_client

client = authenticate_client(key, endpoint)

In [17]:
def sentiment_analysis_example(client, tf):

    document = []
    
    with open(tf, 'r') as f:
        for line in f:
            line = line.strip()
            line = line.replace(u'\xa0', u' ')
            document.append(line)
    f.close()

    response = client.analyze_sentiment(inputs=document)[0]
    return((response.sentiment, response.sentiment_scores.positive, response.sentiment_scores.negative))

In [18]:
# this program will expect the text files to be analyzed 
# to be located an a subdirectory named 'Text'

textdir = os.getcwd()                                  # temproarily disabled for testing -> os.path.abspath("Text")

returned_sentiment = []
returned_sentiment_headers = ("President.Year", "Sentiment", "Pos", "Neg")
returned_sentiment.append(returned_sentiment_headers)

with os.scandir(textdir) as entries:
    for entry in entries:
        if entry.is_file():
            entry_str = str(entry)
            # parse the returned scandir entry
            trimmed_entry_str = entry_str[(entry_str.find("'")+1):entry_str.rfind("'")]
            entry_name = trimmed_entry_str[0:trimmed_entry_str.find('.')]
            entry_ext = trimmed_entry_str[trimmed_entry_str.find(".")+1:]
            # half a$$ed attempt to only read 'president.1234' files
            if entry_ext.isnumeric() and 1789 <= int(entry_ext) <=2100:     # if we last that long
                aFile = textdir + "/" + entry_name + '.' + entry_ext
                this_record = (entry_name + '.' + entry_ext, sentiment_analysis_example(client, aFile))
                returned_sentiment.append(this_record)

print(returned_sentiment)

[('President.Year', 'Sentiment', 'Pos', 'Neg'), ('obama.2016', ('neutral', 0.13, 0.02)), ('trump.2020', ('neutral', 0.23, 0.06))]
